In [6]:
import cnn_builder as cbuild
import config
import copy
import glob
import importlib
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.liver_metrics as lmet
import niftiutils.transforms as tr
import niftiutils.registration as reg
import niftiutils.masks as masks
import numpy as np
import os
from os.path import *
import shutil
import SimpleITK as sitk
import sys
from skimage.morphology import ball
%matplotlib inline

In [2]:
importlib.reload(hf)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

### Corrupt data

In [387]:
big_path = r"C:\Users\Clinton\Desktop\bigpath"
header = hf.dcm_load_header(big_path)

In [409]:
dcm_path = [r"C:\Users\Clinton\Desktop\New folder",
        r"C:\Users\Clinton\Desktop\New folder2",
        r"C:\Users\Clinton\Desktop\New folder3",
        r"C:\Users\Clinton\Desktop\New folder4"]

In [407]:
os.makedirs(dcm_path)
os.makedirs(dcm_path2)
os.makedirs(dcm_path3)
os.makedirs(dcm_path4)

In [344]:
wtf_path = r"D:\Lipiodol\Images all\07-AN\#1 9-10-13 MRI baseline\MRI Abdomen W WO Contrast"

In [376]:
acq_t = [82031.93, 82100.41, 82147.77, 82334.49]

In [410]:
for ix,fn in enumerate(glob.glob(join(wtf_path, "*"))):
    h = hf.dcm_load_header(fn)
    for i in range(4):
        if abs(float(h[0].AcquisitionTime) - acq_t[i]) < .02:
            shutil.copy(fn, join(dcm_path[i], basename(fn)))

for fn in glob.glob(join(dcm_path, "*")):
    if int(header[int(fn[-7:-4])-1].TemporalPositionIdentifier) == 2:
        shutil.move(fn, join(dcm_path2, basename(fn)))
    elif int(header[int(fn[-7:-4])-1].TemporalPositionIdentifier) == 3:
        shutil.move(fn, join(dcm_path3, basename(fn)))
    elif int(header[int(fn[-7:-4])-1].TemporalPositionIdentifier) == 4:
        shutil.move(fn, join(dcm_path4, basename(fn)))

In [ ]:
dim = 1
base = float(header[0].SliceLocation)
for x,h in enumerate(header[1:]):
    dz = float(h.SliceLocation) - base
    print(dz)
    base = float(h.SliceLocation)

In [412]:
fn = ["t1-pre.nii", "t1-art.nii", "t1-pv.nii", "t1-equ.nii"]

In [418]:
for i in range(2,3):
    a,dims = hf.dcm_load(dcm_path[i], True, True)
    save_path = join(dirname(dcm_path[i]), "07", fn[i])
    hf.save_nii(a, save_path, dims)

### MRI loading

In [430]:
patient_id = "09"
modality = "MRI-BL"
dcm_paths = [x for x in glob.glob(join(target_dir, patient_id, modality,"*")) if x.find('.')==-1]
for dcm_path in dcm_paths:
    if not exists(splitext(dcm_path)[0] + ".nii"):
        try:
            save_path = splitext(dcm_path)[0] + ".nii"
            img, dims = hf.dcm_load(dcm_path, True, True)
            hf.save_nii(img, save_path, dims)
        except:
            print(dcm_path)

In [130]:
paths = [x for x in glob.glob(join(target_dir, patient_id, modality,"*.nii.gz")) if '20s' in x or 'Ph1-Ax' in x]
if len(paths)==1:
    os.rename(paths[0], join(dirname(paths[0]), "mri_30d_art.nii.gz"))

In [131]:
paths = [x for x in glob.glob(join(target_dir, patient_id, modality,"*.nii.gz")) \
         if ('pre' in x.lower() and 'post' not in x.lower()) or basename(x).startswith('Ax LAVA-XV')]
if len(paths)==1:
    os.rename(paths[0], join(dirname(paths[0]), "mri_30d_pre.nii.gz"))

## Make liver mask

In [145]:
import keras
from config import Config
model = keras.models.load_model(join(Config().model_dir, "mri_liver.hdf5"))

In [ ]:
lm.seg_liver_mri_from_path(mri30d_art_path, mri30d_liver_mask_path, model, mri30d_tumor_mask_path)

## Non-rigid reg

In [464]:
importlib.reload(reg)

<module 'niftiutils.registration' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\registration.py'>

In [14]:
print(mask_dir)

D:\Lipiodol\Data\12\masks


In [18]:
patient_id = "12"
mask_dir = join(target_dir, patient_id, "masks")
nii_dir = join(target_dir, patient_id, "nii_files")
if not exists(nii_dir):
    os.makedirs(nii_dir)

ct24_path = join(target_dir, patient_id, "nii_files", "ct24.nii.gz")
ct24_tumor_mask_path = glob.glob(join(mask_dir, "tumor*24h*.ids"))[0]

mribl_art_path = join(target_dir, patient_id, "MRI-BL", "mri_bl_art.nii.gz")
mribl_pre_path = join(target_dir, patient_id, "MRI-BL", "mri_bl_pre.nii.gz")
mribl_tumor_mask_path = join(mask_dir, "tumor_BL_MRI")
mribl_liver_mask_path = join(mask_dir, "mri_bl_liver")
mribl_enh_mask_path = join(mask_dir, "enh_bl")
mribl_nec_mask_path = join(mask_dir, "nec_bl")
ct24_bl_enh_mask_path = join(mask_dir, "ct24_bl_enh")
ct24_bl_nec_mask_path = join(mask_dir, "ct24_bl_nec")

mri30d_art_path = join(target_dir, patient_id, "MRI-30d", "mri_30d_art.nii.gz")
mri30d_pre_path = join(target_dir, patient_id, "MRI-30d", "mri_30d_pre.nii.gz")
mri30d_tumor_mask_path = join(mask_dir, "tumor_30dFU_MRI")
mri30d_liver_mask_path = join(mask_dir, "mri_30d_liver")
mri30d_enh_mask_path = join(mask_dir, "enh_30d")
mri30d_nec_mask_path = join(mask_dir, "nec_30d")
ct24_30d_enh_mask_path = join(mask_dir, "ct24_30d_enh")
ct24_30d_nec_mask_path = join(mask_dir, "ct24_30d_nec")

In [133]:
ct_img, ct_dims = hf.dcm_load(join(target_dir, patient_id, "CT24h"), True, True)
hf.save_nii(ct_img, ct24_path, ct_dims)

In [141]:
threshold = 23.148
lmet.seg_tumor_from_threshold(mribl_art_path, mribl_pre_path,
                              threshold, mribl_tumor_mask_path,
                              mribl_enh_mask_path, mribl_nec_mask_path)
threshold = 78.243
lmet.seg_tumor_from_threshold(mri30d_art_path, mri30d_pre_path,
                              threshold, mri30d_tumor_mask_path,
                              mri30d_enh_mask_path, mri30d_nec_mask_path)

#qEASLy to produce enhancing mask
#importlib.reload(lmet)
_=lmet.segment_tumor_from_paths(mribl_art_path, mribl_pre_path,
                              mribl_liver_mask_path, mribl_tumor_mask_path,
                              mribl_enh_mask_path, mribl_nec_mask_path)
_=lmet.segment_tumor_from_paths(mri30d_art_path, mri30d_pre_path,
                              mri30d_liver_mask_path, mri30d_tumor_mask_path,
                              mri30d_enh_mask_path, mri30d_nec_mask_path)

In [17]:
xform_path, crops, pads = reg.get_mask_Tx(ct24_path, ct24_tumor_mask_path, mribl_art_path, mribl_tumor_mask_path)

# Transform using existing transform
_ = reg.transform_mask(mribl_enh_mask_path, mribl_art_path, xform_path,
                     crops, pads, ct_dims, ct24_bl_enh_mask_path)
_ = reg.transform_mask(mribl_nec_mask_path, mribl_art_path, xform_path,
                     crops, pads, ct_dims, ct24_bl_nec_mask_path)

In [143]:
xform_path, crops, pads = reg.get_mask_Tx(ct24_path, ct24_tumor_mask_path, mri30d_art_path, mri30d_tumor_mask_path)

# Transform using existing transform
_ = reg.transform_mask(mri30d_enh_mask_path, mri30d_art_path, xform_path,
                     crops, pads, ct_dims, ct24_30d_enh_mask_path)
_ = reg.transform_mask(mri30d_nec_mask_path, mri30d_art_path, xform_path,
                     crops, pads, ct_dims, ct24_30d_nec_mask_path)

In [90]:
ball_ct24_path = join(target_dir, patient_id, "nii_files", "ct24_ball.nii")
ball_mask_path = join(mask_dir, "ball_mask")
ball_mribl_enh_mask_path = join(mask_dir, "ball_mribl_enh_mask")
ball_mri30d_enh_mask_path = join(mask_dir, "ball_mri30d_enh_mask")

In [91]:
importlib.reload(reg)

xform_path, crops, pads = reg.get_mask_Tx_shape(ct24_path, ct24_tumor_mask_path, mask_path=ball_mask_path)
img = reg.transform_region(ct24_path, xform_path, crops, pads, [1.]*3, ball_ct24_path)

xform_path, crops, pads = reg.get_mask_Tx_shape(mribl_art_path, mribl_tumor_mask_path)
_ = reg.transform_mask(mribl_enh_mask_path, mribl_art_path, xform_path,
                     crops, pads, [1.]*3, ball_mribl_enh_mask_path)

#xform_path, crops, pads = reg.get_mask_Tx_shape(mri30d_art_path, mri30d_tumor_mask_path)
#_ = reg.transform_mask(mri30d_enh_mask_path, mri30d_art_path, xform_path,
#                     crops, pads, [1.]*3, ball_mri30d_enh_mask_path)

In [108]:
importlib.reload(reg)
xform_path, crops, pads = reg.get_mask_Tx_shape(mribl_art_path, mribl_tumor_mask_path)
_ = reg.transform_mask(mribl_tumor_mask_path, mribl_art_path, xform_path,
                     crops, pads, [1.]*3, ball_mribl_enh_mask_path)

In [86]:
img, dims = hf.nii_load("D:\\temp_fix.nii")

### Compose with rotation

In [191]:
mribl_liver_mask_path = join(target_dir, patient_id, "liver bl final.ids")
ct24_liver_mask_path = glob.glob(join(target_dir, patient_id, "wholeliver_24hCT*.ids"))[0]

In [ ]:
mribl_ct24_xform, ct24_tumor, mribl_tumor, mribl_ct24_crops = full_reg_masks(ct24_tumor_mask_path, ct24_path, mribl_tumor_mask_path, mribl_art_path)

In [ ]:
# BL MRI to 24h CT
vec_field = lm.reg_masks(ct_tumor_mask_path, blmri_tumor_mask_path)

In [ ]:
mribl_enh_mask = reg.apply_field(vec_field)

In [ ]:
ct24_lip = lm.seg_lipiodol()

In [ ]:
reg.reg_sitk(ct24_lip, vec_field)

In [114]:
mri_dcm_paths = [x for x in glob.glob(join(target_dir, '*', 'MRI-BL', '*20s*')) if '.nii' not in x]
mri_dcm_paths

In [ ]:
path_dict = {}
patient_id = '04'
for path in glob.glob(join(target_dir, patient_id, "MRI-BL","*","*")):
    header = hf.dcm_load_header(path)
    if len(header) > 0:
        series_descript = header[0][('0008', '103e')].value
        path_dict[path] = series_descript
        series_descript = series_descript.replace("/","-")
        os.rename(path, join(target_dir,patient_id, "MRI-BL", series_descript))

In [499]:
for path in glob.glob(join(target_dir, "*", "masks","*final*")):
    if not exists(path.replace(" final", "")):
        os.rename(path, path.replace(" final", ""))

for path in glob.glob(join(target_dir, "*")):
    os.makedirs(join(path, "masks"))

for path in glob.glob(join(target_dir, "*", "*.off")):
    os.rename(path, join(dirname(path), "masks", basename(path)))

### Bias field corr

In [122]:
mri_raw_paths = glob.glob(join(target_dir, '*', 'MRI-BL', 't1-art.nii.gz'))
mri_bfc_paths = [join(dirname(x), "t1-art-bfc.nii.gz") for x in mri_raw_paths]

In [176]:
importlib.reload(masks)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [ ]:
glob.glob(join(target_dir, patient_id, ''

In [ ]:
masks.mask2nii()

In [128]:
inputImage = sitk.ReadImage(mri_raw_paths[0])
maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

if len ( sys.argv ) > 3:
    inputImage = sitk.Shrink( inputImage, [ int(sys.argv[3]) ] * inputImage.GetDimension() )
    maskImage = sitk.Shrink( maskImage, [ int(sys.argv[3]) ] * inputImage.GetDimension() )

inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )

corrector = sitk.N4BiasFieldCorrectionImageFilter()

numberFilltingLevels = 4
#corrector.SetMaximumNumberOfIterations( [ int( sys.argv[5] ) ] *numberFilltingLevels  )

output = corrector.Execute( inputImage, maskImage )

sitk.WriteImage( output, mri_bfc_paths[0] )

### Masks

## Lipiodol seg

In [82]:
importlib.reload(lm)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [144]:
# Lipiodol 3 masks
info = {}
for seg in glob.glob(join(mask_dir, "wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(target_dir, patient_id, "CT24h"), True, True)
        mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)
        
    lm.seg_lipiodol((mask > 0) * ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims, mask)
    break

In [ ]:
# Lipiodol target vs non-target masks
info = {}
for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)
        
    lm.seg_target_lipiodol((mask > 0) * ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims, mask)
    #break